In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [6]:
load_dotenv()

llm = ChatOpenAI()

In [7]:

class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [8]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [9]:

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [10]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [11]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!',
 'explanation': 'This joke is a play on words using the word "crusty." In this case, crusty can refer to both the crust of a pizza and also to someone feeling unwell or not their best. So the joke is saying that the pizza went to the doctor because it was feeling unwell, and in this case, the reason for feeling unwell is because it was "crusty." The humor comes from the unexpected double meaning of the word "crusty" in the context of a pizza going to the doctor.'}

In [12]:
workflow.get_state(config1)


StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!', 'explanation': 'This joke is a play on words using the word "crusty." In this case, crusty can refer to both the crust of a pizza and also to someone feeling unwell or not their best. So the joke is saying that the pizza went to the doctor because it was feeling unwell, and in this case, the reason for feeling unwell is because it was "crusty." The humor comes from the unexpected double meaning of the word "crusty" in the context of a pizza going to the doctor.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-8149-6ccd-8002-3537a5191024'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-17T09:02:26.294087+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-66cc-609e-8001-5c2aa0c83ab7'}}, tasks=(), interrupts=())

In [13]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!', 'explanation': 'This joke is a play on words using the word "crusty." In this case, crusty can refer to both the crust of a pizza and also to someone feeling unwell or not their best. So the joke is saying that the pizza went to the doctor because it was feeling unwell, and in this case, the reason for feeling unwell is because it was "crusty." The humor comes from the unexpected double meaning of the word "crusty" in the context of a pizza going to the doctor.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-8149-6ccd-8002-3537a5191024'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-17T09:02:26.294087+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-66cc-609e-8001-5c2aa0c83ab7'}}, tasks=(), interrupts=()),
 StateSnapsho

In [14]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta chef go broke? Because he pasta-way all his dough on noodles!',
 'explanation': 'This joke is a play on words using pasta-related puns. The phrase "pasta-way all his dough" is a reference to the pasta chef spending all of his money on noodles, which in this context is a humorous way of saying he spent all his money on pasta ingredients. The joke is funny because it combines the culinary world with financial troubles in a clever and punny way.'}

In [14]:
workflow.get_state(config2)


StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta join a band? \n\nBecause it had great orecchiette solos!', 'explanation': 'This joke plays on the double meaning of "join a band." In this case, joining a band can refer to joining a group of musicians, but it can also mean to physically join together, like pasta does when it is cooked. The punchline about the pasta having great orecchiette solos is a play on words, as "orecchiette" is a type of pasta known for its unique shape that resembles a small ear. The joke suggests that the pasta joined the band because it had excellent orecchiette solos, as if the pasta itself was a talented musician.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f093a36-9b9d-662c-8002-02dee6e9b7eb'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-17T08:50:51.122436+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f093a

In [15]:
list(workflow.get_state_history(config2))


[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta join a band? \n\nBecause it had great orecchiette solos!', 'explanation': 'This joke plays on the double meaning of "join a band." In this case, joining a band can refer to joining a group of musicians, but it can also mean to physically join together, like pasta does when it is cooked. The punchline about the pasta having great orecchiette solos is a play on words, as "orecchiette" is a type of pasta known for its unique shape that resembles a small ear. The joke suggests that the pasta joined the band because it had excellent orecchiette solos, as if the pasta itself was a talented musician.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f093a36-9b9d-662c-8002-02dee6e9b7eb'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-17T08:50:51.122436+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f093

## Time Travel

In [15]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f093a50-5815-649f-8000-4deadefc980b"}})


StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f093a50-5815-649f-8000-4deadefc980b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-17T09:02:21.973417+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-580b-684f-bfff-7de818ccf220'}}, tasks=(PregelTask(id='64e0010d-e045-bd0d-73aa-492fa233ba03', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!'}),), interrupts=())

In [16]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f093a50-5815-649f-8000-4deadefc980b"}})


{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!',
 'explanation': 'This joke is a pun based on the words "crusty" and "crust." In this context, "crusty" is often used to describe something that is dry, rough, and stiff. In this joke, the pizza went to the doctor because it was feeling "crusty," which can refer to the texture of the pizza\'s crust. This play on words adds humor to the idea of a pizza seeking medical attention.'}

In [17]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!', 'explanation': 'This joke is a pun based on the words "crusty" and "crust." In this context, "crusty" is often used to describe something that is dry, rough, and stiff. In this joke, the pizza went to the doctor because it was feeling "crusty," which can refer to the texture of the pizza\'s crust. This play on words adds humor to the idea of a pizza seeking medical attention.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a57-06b8-661c-8002-68cd35b3ce1b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-17T09:05:21.346716+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a56-f81a-6c1e-8001-295ff76033d3'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was

## Update State

In [21]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f093a50-5815-649f-8000-4deadefc980b", "checkpoint_ns": ""}}, {'topic':'samosa'})


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f093a5f-67d6-631d-8001-c2f45b510a65'}}

In [22]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a5f-67d6-631d-8001-c2f45b510a65'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-09-17T09:09:06.278479+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a50-5815-649f-8000-4deadefc980b'}}, tasks=(PregelTask(id='5fecef17-e426-ef90-bc1b-fc0a8d1fd291', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza maker go broke?\nHe couldn't make enough dough!", 'explanation': 'This joke plays on the double meaning of the word "dough." In the context of pizza making, "dough" refers to the mixture of flour, water, and yeast that is used to make the pizza crust. However, "dough" is also a slang term for money. So, the pun

In [23]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f093a5f-67d6-631d-8001-c2f45b510a65"}})


{'topic': 'samosa',
 'joke': 'Why did the samosa go to school? To get a little more dough!',
 'explanation': 'This joke plays on the double meaning of the word "dough." In this context, "dough" refers to money, but it also refers to the bread-like substance used to make samosas. So, the joke is suggesting that the samosa went to school to get more money (dough) but also to physically get more dough to make itself bigger. It\'s a play on words that adds a humorous twist to the idea of a samosa going to school.'}

In [24]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to school? To get a little more dough!', 'explanation': 'This joke plays on the double meaning of the word "dough." In this context, "dough" refers to money, but it also refers to the bread-like substance used to make samosas. So, the joke is suggesting that the samosa went to school to get more money (dough) but also to physically get more dough to make itself bigger. It\'s a play on words that adds a humorous twist to the idea of a samosa going to school.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a61-9776-6d1d-8003-c9af10ae57ef'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-17T09:10:04.959670+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f093a61-8a15-6f9e-8002-0f590ef6abe8'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go

## Fault Tolerance

In [2]:

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [3]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [5]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:

try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:

# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...


NameError: name 'graph' is not defined

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))